In [16]:
import pandas as pd
import numpy as np
import os
import sys
import sqlite3

In [35]:
pd.set_option('display.float_format', lambda x: '{:,.0f}'.format(x).replace(',', ' '))

db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')

conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data = pd.read_sql_query(query, conn)

In [30]:
from scripts.preprocess_final import create_complete_pipeline, create_preprocessing_pipeline_init

In [19]:
data = data.drop(columns='montant')

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299670 entries, 0 to 299669
Data columns (total 31 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   uid                        299670 non-null  object 
 1   id                         299670 non-null  object 
 2   nature                     299670 non-null  object 
 3   acheteur_id                299670 non-null  object 
 4   acheteur_nom               299477 non-null  object 
 5   acheteur_siren             299500 non-null  object 
 6   titulaire_id               299670 non-null  object 
 7   titulaire_typeIdentifiant  299670 non-null  object 
 8   titulaire_nom              292097 non-null  object 
 9   titulaire_siren            298537 non-null  object 
 10  objet                      299670 non-null  object 
 11  montant                    299670 non-null  float64
 12  codeCPV                    299670 non-null  object 
 13  procedure                  29

In [42]:
pipeline = create_complete_pipeline(cat= 'marche_sim', top_n=30, level=2)
pipeline


Pipeline(steps=[('init',
                 Pipeline(steps=[('id_accord_encoder', IdAccordCadreEncoder()),
                                 ('taux_avance_categorizer',
                                  TauxAvanceCategorizer()),
                                 ('outliers_feature_rows_selector',
                                  InitTransformer(cat='marche_sim',
                                                  top_n=30))])),
                ('follow',
                 Pipeline(steps=[('preprocessing',
                                  Pipeline(steps=[('duree_mois_dropper',
                                                   DureeMoisDropper()),
                                                  ('column_transformer',
                                                   ColumnTra...
                                                                                                                   strategy='constant')),
                                                                                                    ('string_converter',
                                                                                                     StringConverter()),
                                                                                                    ('onehot',
                                                                                                     OneHotEncoder(handle_unknown='ignore',
                                                                                                                   sparse_output=False))]),
                                                                                    ['nature',
                                                                                     'procedure',
                                                                                     'formePrix',
                                                                                     'ccag',
                                                                                     'typeGroupementOperateurs',
                                                                                     'codeCPV_2',
                                                                                     'tauxAvance_cat'])]))])),
                                 ('to_dataframe',
                                  FunctionTransformer(func=<function create_preprocessing_pipeline_follow.<locals>.<lambda> at 0x7f2c261343a0>))]))])

In [36]:
pipeline2 = create_preprocessing_pipeline_init(cat='pred_montant' , min=20000, max=50000000, top_n=40, level=2)


In [40]:
data_processed = pipeline.fit_transform(data)
data_processed

,offres_recues_pipeline__offresRecues,other_num_pipeline__montant,other_num_pipeline__dureeMois,other_num_pipeline__origineFrance,binary_pipeline__marcheInnovant,binary_pipeline__sousTraitanceDeclaree,binary_pipeline__idAccordCadre,cat_pipeline__nature_Marché,cat_pipeline__nature_Marché de défense ou de sécurité,cat_pipeline__nature_Marché de partenariat,...,cat_pipeline__codeCPV_2_79000000,cat_pipeline__codeCPV_2_80000000,cat_pipeline__codeCPV_2_85000000,cat_pipeline__codeCPV_2_90000000,cat_pipeline__codeCPV_2_92000000,cat_pipeline__tauxAvance_cat_large_advance,cat_pipeline__tauxAvance_cat_medium_advance,cat_pipeline__tauxAvance_cat_missing,cat_pipeline__tauxAvance_cat_no_advance,cat_pipeline__tauxAvance_cat_small_advance
0,-0,-1,-1,-0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-1,-1,1,-0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-0,2,0,-0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-1,2,2,-0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-1,1,1,-0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276923,2,-0,1,-0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
276924,0,-1,-1,-0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
276925,1,2,-0,-0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
276926,-0,1,0,-0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
